## 모델 Parameter

### 손실 함수 (Loss Function)
- 예측 값과 실제 값 사이의 오차를 측정
- 학습이 진행되면서 해당 학습 과정이 얼마나 잘 진행되고 있는지 나타내는 지표
- 모델이 훈련되는 동안 최소화 시켜야할 값으로 주어진 문제에 대한 성공 지표
- 손실 함수에 따른 결과를 통해 학습 파라미터를 조정
- 최적화 이론에서 최소화하고자 하는 함수
- 미분 가능한 함수 (실제로 convex할 때 optima가 존재하고, 찾을 수 있기 때문)
- torch의 주요 loss function
    - `torch.nn.BCELoss` : 이진 분류를 위해 사용
    - `torch.nn.CrossEntropyLoss` : 다중 클래스 분류를 위해 사용
    - `torch.nn.MSELoss` : 회귀 모델에서 사용 
    - `torch.nn.L1Loss` : 다른 loss와 더해져서 regularization 효과를 줄 때 사용
    - `torch.nn.KLDivLoss` : KL-Divergence를 계산해줌 (어떤 두 분포의 차이를 볼 때 사용)

### Optimizer
- 손실 함수를 기반으로 모델이 어떻게 update 되어야하는지 결정 (Gradient Descent를 수행하는 특정 알고리즘)
- optimizer는 `step()`을 통해 전달받은 모델의 파라미터를 update함
- 모든 optimizer의 기본은 `torch.optim.Optimizer(params, defaults) 클래스를 사용함
- `zero_grad()` 를 이용하여 optimizer가 update할 파라미터들의 기울기를 0으로 설정
- `torch.optim.lr_scheduler`를 이용해 Epochs에 따라 learning rate를 조절할 수 있음
- torch의 주요 optimizer
    - `torch.optim.Adagrad()` : *Adaptive Subgradient Methods for Online Learning and Stochastic Optimization*
    - `torch.optim.Adam()` : *Adam: A Method for Stochastic Optimization*
    - `torch.optim.RMSprop()` : *G.Hinton's Lecture course*
    - `torch.optim.SGD()` : *Stochastic gradient descent*

<img src=./images/Optimizer.gif>

## Learning rate Scheduler
- 학습 시 특정 조건에 따라 learning rate을 조정하면서 최적화를 수행
- 일정 횟수 이상이 되면 learning rate decay(감소) / global minima 근처에 가면 learning rate 줄이기

### torch의 learning rate scheduler
- `optim.lr_scheduler.LambdaLR()`
    - 파이썬의 람다함수를 이용해 그 결과를 learning rate으로 설정

    - lambda1 = lambda epoch: epoch // 30

    - scheduler = LambdaLR(optimizer, lr_lambda=[lambda1])


- `optim.lr_scheduler.StepLR()`
    - step마다 learning rate을 gamma ratio 만큼 decay

    - scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    
    - $epoch < 30 -> lr = 0.01, 30 < epoch < 60 -> lr = 0.001$ 

- `optim.lr_scheduler.MultiStepLR()`
    - 위의 `StepLR()`과 비슷하지만 milestone을 지정해주어 특정 epoch에 gamma ratio 만큼 decay
    
    - scheduler = MultiStepLR(optimizer, milestones=[30, 50, 100], gamma=0.1)

- `optim.lr_scheduler.ExponentialLR()`
    - 매 epoch마다 learning rate * gamma 만큼 곱함

    - 자주 사용되는 건 못 봄

- `optim.lr_scheduler.CosineAnnealingLR()`
    - *SGDR: Stochastic Gradient Descent with Warm Restarts*

    - learning rate을 cosine 함수 형태처럼 변화시켜 학습률이 커질 때도 있고, 작아질 때도 있음

- `optim.swa_utils`
    - 2018, UAI *Averaging Weights Leads to Wider Optima and Better Generalizaion*

    - 효과적이라서 torch 1.6 부터 공식적으로 지원해줌

    - `from torch.optim.swa_utils import AverageModel, SWALR`

    - `swa_model = AveragedModel(model)` - to compute the weights of the SWA model

    - `swa_model.update_parameters(model)` - to update averages

    - `swa_scheduler = SWALR(optimizer, anneal_strategy='Linear', ...)`

    - `annel_strategy='cos'`로 하면 cosine annealing으로 사용할 수 있음

    - 학습이 끝난 후에는 `swa_utils.update_bn()`으로 batch normalization statistics update